In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl

%matplotlib inline

# better pictures and legends
plt.rc('figure', dpi=250)
plt.rc('text', usetex=True)

In [ ]:
from astropy import units as u, constants as const
import numpy as np

class GU():
    def __init__(self, v):
        self.v = v*u.Msun
        self.M = self._to(u.g)
        self.L = self._to(u.cm)
        self.T = self._to(u.s)
        self.Density = self.MLT(1, -3, 0)
        self.Energy = self.MLT(1, 2, -2)
        self.MagneticFluxDensity = self.MLT(1/2, -1/2, -1)
        self.Luminosity = self.Energy/self.T
    def _to(self, unit):
        if unit.is_equivalent(u.Msun):
            return self.v.to(unit)
        if unit.is_equivalent(u.cm):
            return (self.v*const.G/const.c**2).to(unit)
        if unit.is_equivalent(u.s):
            return (self.v*const.G/const.c**3).to(unit)
    def MLT(self, m, l, t):
        return self.M**m * self.L**l * self.T**t

In [ ]:
def time_text(time, mass, merger) -> str:
    t = GU(mass).T.to('ms').value * (time - merger)
    if t < 0:
        return f'$\mathsf{{{-t:4.0f} ms before merger}}$'.replace(' ', '\ ')
    return f'$\mathsf{{{-t:4.0f} ms after merger}}$'.replace(' ', '\ ')

print(time_text(7777, 30+15, 7789))

In [ ]:
directory = "/home/enlil/markin/rose/states/espl_200_fives_viridis_0.25lwidth_horizons"

first_frame = plt.imread(f'{directory}/frame{0:06d}.png')
print(first_frame.shape)


px = 1/plt.rcParams['figure.dpi'] 
figsize = (first_frame.shape[1]*px, first_frame.shape[0]*px)

print(figsize)


start_time = 0.0
end_time = 8500
number_of_frames = 1200
merger = 7789
mass = 30 + 15

r_domain = 500 * GU(mass).T.to('ms').value
print(r_domain)

frame_times = np.linspace(start_time, end_time, number_of_frames)

for frame_id, frame_time in enumerate(frame_times): 
    print(f'Processing frame #{frame_id}')
    fig = plt.figure(figsize=figsize)
    ax = fig.gca()
    fig.tight_layout()

    frame = plt.imread(f'{directory}/frame{frame_id:06d}.png')

    ax.imshow(frame)
    ax.margins(0, 0)
    ax.axis('off')


    # Description
    ax.annotate(r'$\mathsf{M = 30\ M_{\odot} + 15\ M_{\odot}}$',
        xy=(0.02, 0.98), xycoords='axes fraction', fontsize=14,
        xytext=(5, -10), textcoords='offset points',
        color = 'white'
    )

    ax.annotate(r'$\mathsf{D_L = 300\ Mpc}$',
        xy=(0.02, 0.95), xycoords='axes fraction', fontsize=14,
        xytext=(5, -10), textcoords='offset points',
        color = 'white'
    )

    ax.annotate(f'$\mathsf{{R_{{domain}} = {r_domain:.0f}\ ms}}$',
        xy=(0.02, 0.92), xycoords='axes fraction', fontsize=14,
        xytext=(5, -10), textcoords='offset points',
        color = 'white'
    )


    norm = mpl.colors.Normalize(vmin=40, vmax=120)
    mappable = mpl.cm.ScalarMappable(norm=norm, cmap='viridis')
    cbaxes = ax.inset_axes(bounds=[0.94, 0.035, 0.01, 0.4])
    cb = fig.colorbar(mappable, cax=cbaxes, orientation='vertical')
    cbaxes.yaxis.label.set_color('black')
    cbaxes.tick_params(axis='y', colors='white', labelsize=12)
    cbaxes.yaxis.set_major_formatter(lambda x, pos: f'$\mathsf{{{x:.0f}}}$')
    for spine in cbaxes.spines:
        cbaxes.spines[spine].set_color('#3e3e3e')
    ax.annotate("$\mathsf{eSPL\ [dB]}$",
        xy=(0.915, 0.46), xycoords='axes fraction', fontsize=14,
        xytext=(5, 10), textcoords='offset points',
        color = 'white'
    )


    
    ax.annotate(time_text(frame_time, mass=mass, merger=merger),
        xy=(0.02, 0.02), xycoords='axes fraction', fontsize=14,
        xytext=(5, 10), textcoords='offset points',
        color = 'white'
    )


    fig.savefig(f'{directory}/frame-pp{frame_id:06d}.png', pad_inches=0, bbox_inches='tight')
    plt.close(fig)